In [1]:
import os
import sys
import logging
import pandas as pd
import numpy as np
import yaml

sys.path.append(os.path.dirname(os.path.abspath('')))
#from utils.spark import restart_spark
from stratification.params import SplitBuilderParams
from prepilot_local.params import PrepilotParams
from prepilot_local.prepilot_experiment_builder import PrepilotExperimentBuilder
from prepilot_local.prepilot_split_builder import PrepilotSplitBuilder
from analysis.abtest import ABTest

logging.basicConfig(level = logging.INFO)

%load_ext autoreload
%autoreload 2

In [2]:
#df = pd.read_csv("TLO.csv", sep=';')

In [3]:
#df.rename(columns={"moda_city":"region","pre_gp_orders":"rto"},inplace=True)

In [4]:
#df["pre_gp_orders"] = df["pre_gp_orders"].str.replace(',', '.').astype(float)

In [5]:
df = pd.read_csv('ab_data.csv')

with open("../analysis/configs/config.yaml", "r") as stream:
    try:
        ab_config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [6]:
df["moda_city"] = np.random.randint(1, 5, df.shape[0])
df["moda_city"] = df["moda_city"].astype(str)
df["id"] = df.index

In [7]:
#DATE_FROM = datetime.date(2021, 4, 1)  # CHANGE
#DATE_TO = datetime.date(2021, 4, 15)  # CHANGE
#synthetic_catalog_ids = []  # List[int]
#CAMPAIGN_ID = "CVM-0-0"  # CHANGE - for naming final output files


split_builder_params = SplitBuilderParams(
    map_group_names_to_sizes={
        'control': None,
        'target': None
    },
    region_col = "moda_city",
    split_metric_col = "height_now",
    customer_col = "id",
    cols = [],
    cat_cols=[
        #'offer_rk_goal',
        #'offer_rk_campaign'
    ],
    pvalue=0.05,
    n_top_cat=100,
    stat_test="ttest_ind"
)

In [8]:
prepilot_params = PrepilotParams(
    #datestart=datestart,
    #datepostperiod=datepostperiod,
    metrics_names=['height_now'],
    injects=[1.000001 ,1.00001, 1.0001, 1.001],#[1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1,1.15,1.2,1.25],
    min_group_size=50000, 
    max_group_size=100000, 
    step=50000,
    variance_reduction = ABTest.cuped,
    use_buckets = True,
    bootstrap_metric = np.mean,#np.median
    iterations_number = 10,
    n_buckets = 10000,
    max_beta_score=0.5,
    min_beta_score=0.02,
)

In [9]:
prepilot = PrepilotExperimentBuilder(df, ab_config,
                                     prepilot_params,
                                     split_builder_params)

In [10]:
%%time
beta,alpha = prepilot.collect()

INFO:stratification.split_builder:Calculate stratas for guest table
INFO:stratification.binning:24560 outliers found
INFO:stratification.split_builder:control: Desired size = 0.25,         resulting size = 49998, diff = 0.0 %
INFO:stratification.split_builder:target: Desired size = 0.3333288889481474,         resulting size = 50002, diff = 0.0 %
INFO:stratification.split_builder:Success!
INFO:stratification.split_builder:control: Desired size = 0.25,         resulting size = 49998, diff = 0.0 %
INFO:stratification.split_builder:target: Desired size = 0.3333288889481474,         resulting size = 50002, diff = 0.0 %
INFO:stratification.split_builder:Success!
INFO:stratification.split_builder:control: Desired size = 0.25,         resulting size = 49998, diff = 0.0 %
INFO:stratification.split_builder:target: Desired size = 0.3333288889481474,         resulting size = 50002, diff = 0.0 %
INFO:stratification.split_builder:Success!
INFO:stratification.split_builder:control: Desired size = 0.2

CPU times: user 19.4 s, sys: 534 ms, total: 20 s
Wall time: 22.7 s


In [11]:
beta

split_rate          (50000, 50000) (100000, 100000)
metric     MDE                                     
height_now 1.000001          >=0.5            >=0.5
           1.000010          >=0.5            >=0.5
           1.000100          >=0.5              0.3
           1.001000         <=0.02           <=0.02

In [12]:
alpha

split_rate,"(50000, 50000)","(100000, 100000)"
metric,,
height_now,0.000000,0.100000
